##### x=texts
##### y=max_label (19位標註者標注最多的標籤)

In [1]:
import pandas as pd
# dataset = pd.read_csv("./dataset_sentence.csv",encoding='utf-8')
dataset = pd.read_excel(r'./200416_label6_dataset_sentence.xlsx')

In [2]:
dataset[:5]

,id,round,sender,texts,max_label,round_label,labels
0,4227729258237820,1,customer,内涵 段子 联通 皮 点赞 中国联通 中国联通 客服 掌上 营业厅 内涵 段子 话题 封 郑...,3,2,0
1,4227729258237820,2,helpdesk,u,6,4,1
2,4227729258237820,3,customer,夸夸,3,0,0
3,4227729258237820,4,helpdesk,*,6,4,1
4,4121001149457180,1,customer,距离 反映 问题 已经 一个 星期 花粉 助手 D 荣耀 honor 荣耀 手机 华为 终端...,2,2,0


In [6]:
texts = list(dataset["texts"])
labels = list(dataset["max_label"])
#ID = list(dataset["id"])
#Round = list(dataset["round"])
#sender = list(dataset["sender"])

In [7]:
X_train=texts[1755:]
X_test=texts[0:1755]
y_train=labels[1755:]
y_test=labels[0:1755]

In [8]:
y_test[:10]

[3, 6, 3, 6, 2, 4, 0, 4, 0, 4]

In [10]:
import numpy as np
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [23]:
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers.embeddings import Embedding
import numpy as np
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils

tokenizer=Tokenizer(num_words = 20000)  #创建一个Tokenizer对象
#fit_on_texts函数可以将输入的文本中的每个词编号，编号是根据词频的，词频越大，编号越小
tokenizer.fit_on_texts(X_train)
vocab=tokenizer.word_index #得到每个词的编号

X_train_word_ids=tokenizer.texts_to_sequences(X_train)
X_test_word_ids = tokenizer.texts_to_sequences(X_test)

X_train_padded_seqs=pad_sequences(X_train_word_ids,maxlen=150) #将超过固定值的部分截掉，不足的在最前面用0填充
X_test_padded_seqs=pad_sequences(X_test_word_ids, maxlen=150)

y_one_train = np_utils.to_categorical(y_train)
y_one_test = np_utils.to_categorical(y_test)

In [16]:
# import BatchNormalization
from keras.layers.normalization import BatchNormalization
import keras
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics

In [27]:
def CNN_model():
    model = Sequential()
    model.add(Embedding(len(vocab)+1, 300, input_length=150)) #使用Embeeding层将每个词编码转换为词向量
    model.add(Conv1D(2, 2, padding='same', strides = 1, activation='relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(units=256,activation='relu'))
    model.add(Dropout(0.35))
    model.add(Dense(units=128,activation='relu'))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model
#     one_hot_labels = keras.utils.to_categorical(y_train, num_classes=7)  # 将标签转换为one-hot编码
#     model.fit(X_train_padded_seqs, one_hot_labels,epochs=5, batch_size=800)
#     y_predict = model.predict_classes(X_test_padded_seqs)  # 预测的是类别，结果就是类别号
#     y_predict = list(map(str, y_predict))
#     print('accuracy', metrics.accuracy_score(y_test, y_predict))
#     print('f1-score:', metrics.f1_score(y_test, y_predict, average='weighted'))

In [28]:
score = CNN_model()
# one_hot_labels = keras.utils.to_categorical(y_train, num_classes=7)
score.fit(X_train_padded_seqs, y_one_train,epochs=20, batch_size=32, verbose=1, validation_split=0.2)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 150, 300)          6533100   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 150, 2)            1202      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 37, 2)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 74)                0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 74)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               19200     
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)              

//anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 12320 samples, validate on 3080 samples
Epoch 1/20
12320/12320 [==============================] - 30s 2ms/step - loss: 1.1965 - accuracy: 0.5695 - val_loss: 0.9066 - val_accuracy: 0.7032
Epoch 2/20
12320/12320 [==============================] - 30s 2ms/step - loss: 0.9515 - accuracy: 0.6762 - val_loss: 0.8327 - val_accuracy: 0.7192
Epoch 3/20
12320/12320 [==============================] - 31s 2ms/step - loss: 0.8462 - accuracy: 0.7156 - val_loss: 0.8083 - val_accuracy: 0.7341
Epoch 4/20
12320/12320 [==============================] - 31s 2ms/step - loss: 0.7897 - accuracy: 0.7381 - val_loss: 0.8028 - val_accuracy: 0.7331
Epoch 5/20
12320/12320 [==============================] - 30s 2ms/step - loss: 0.7448 - accuracy: 0.7567 - val_loss: 0.8029 - val_accuracy: 0.7377
Epoch 6/20
12320/12320 [==============================] - 30s 2ms/step - loss: 0.7106 - accuracy: 0.7693 - val_loss: 0.8268 - val_accuracy: 0.7302
Epoch 7/20
12320/12320 [==============================] - 31s 3ms/ste

In [29]:
pre_probability = score.predict(X_test_padded_seqs)
cnn_predicted = pre_probability.argmax(axis=-1)

In [30]:
from sklearn import metrics
print("Macro_average: {0}".format(metrics.f1_score(y_test, cnn_predicted, average="macro")))
print("Micro_average: {0}".format(metrics.f1_score(y_test, cnn_predicted, average="micro")))
print(metrics.classification_report(y_test, cnn_predicted))

# print('準確率', metrics.accuracy_score(y_test, y_predict))
# print('平均f1-score:', metrics.f1_score(y_test, y_predict, average='macro'))

Macro_average: 0.3602071807792571
Micro_average: 0.7173789173789173
              precision    recall  f1-score   support

           0       0.58      0.82      0.68       477
           1       0.00      0.00      0.00        23
           2       0.78      0.78      0.78       374
           3       0.00      0.00      0.00       101
           4       0.83      0.85      0.84       662
           5       0.00      0.00      0.00        33
           6       0.79      0.13      0.22        85

    accuracy                           0.72      1755
   macro avg       0.42      0.37      0.36      1755
weighted avg       0.67      0.72      0.68      1755



In [21]:
from keras.layers import LSTM
def LSTM_model():
    model = Sequential()
    model.add(Embedding(output_dim=32,input_dim=20000,input_length=150))
    model.add(Dropout(0.35))
    model.add(LSTM(units=16))
    model.add(Dense(units=256,activation='relu'))
    model.add(Dropout(0.35))
    model.add(Dense(units=128,activation='relu'))
    model.add(Dense(units=7,activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [24]:
lstm = LSTM_model()
lstm.fit(X_train_padded_seqs, y_one_train, epochs=20, batch_size=128, verbose=1, validation_split=0.2)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 150, 32)           640000    
_________________________________________________________________
dropout_5 (Dropout)          (None, 150, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense_6 (Dense)              (None, 256)               4352      
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_8 (Dense)              (None, 7)                

//anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 12320 samples, validate on 3080 samples
Epoch 1/20
12320/12320 [==============================] - 7s 554us/step - loss: 1.3013 - accuracy: 0.5333 - val_loss: 0.7841 - val_accuracy: 0.7565
Epoch 2/20
12320/12320 [==============================] - 6s 474us/step - loss: 0.7294 - accuracy: 0.7615 - val_loss: 0.6528 - val_accuracy: 0.7925
Epoch 3/20
12320/12320 [==============================] - 6s 474us/step - loss: 0.5707 - accuracy: 0.8099 - val_loss: 0.6351 - val_accuracy: 0.7984
Epoch 4/20
12320/12320 [==============================] - 6s 453us/step - loss: 0.4657 - accuracy: 0.8395 - val_loss: 0.6574 - val_accuracy: 0.7880
Epoch 5/20
12320/12320 [==============================] - 6s 466us/step - loss: 0.3931 - accuracy: 0.8612 - val_loss: 0.7158 - val_accuracy: 0.7938
Epoch 6/20
12320/12320 [==============================] - 6s 458us/step - loss: 0.3413 - accuracy: 0.8791 - val_loss: 0.7182 - val_accuracy: 0.7857
Epoch 7/20
12320/12320 [==============================] - 6s 47

In [25]:
pre_probability = lstm.predict(X_test_padded_seqs)
lstm_predicted = pre_probability.argmax(axis=-1)

In [26]:
from sklearn import metrics
print("Macro_average: {0}".format(metrics.f1_score(y_test, lstm_predicted, average="macro")))
print("Micro_average: {0}".format(metrics.f1_score(y_test, lstm_predicted, average="micro")))
print(metrics.classification_report(y_test, lstm_predicted))

Macro_average: 0.5231940584734954
Micro_average: 0.741880341880342
              precision    recall  f1-score   support

           0       0.69      0.65      0.67       477
           1       0.32      0.52      0.39        23
           2       0.85      0.88      0.87       374
           3       0.34      0.15      0.21       101
           4       0.84      0.88      0.86       662
           5       0.22      0.21      0.22        33
           6       0.38      0.53      0.45        85

    accuracy                           0.74      1755
   macro avg       0.52      0.55      0.52      1755
weighted avg       0.74      0.74      0.74      1755



In [31]:
MAX_SEQ_LEN = 30

In [32]:
import os
from keras_bert import load_trained_model_from_checkpoint

BERT_PRETRAINED_DIR = '/Users/yihsuanhuang/BERT/chinese_L-12_H-768_A-12' # need to rewrite if you put model in other directory
config_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
checkpoint_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
model = load_trained_model_from_checkpoint(config_file, checkpoint_file, seq_len=MAX_SEQ_LEN)

model.summary(line_length=300)

Model: "model_2"
____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                                                       Output Shape                                                      Param #                             Connected to                                                                                       
Input-Token (InputLayer)                                                                           (None, 30)                                                        0                                                                                                                                      
________________________________________________________________________________

In [33]:
from keras.engine import Layer

class NonMaskingLayer(Layer):   
    def __init__(self, **kwargs):   
        self.supports_masking = True  
        super(NonMaskingLayer, self).__init__(**kwargs)   
  
    def build(self, input_shape):   
        input_shape = input_shape   
  
    def compute_mask(self, input, input_mask=None):   
        # do not pass the mask to the next layers   
        return None   
  
    def call(self, x, mask=None):   
        return x   
  
    def get_output_shape_for(self, input_shape):   
        return input_shape

In [35]:
#CNN
import codecs
import numpy as np
from sklearn.preprocessing import LabelEncoder

from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.layers import Convolution1D, MaxPool1D, Flatten
from keras.models import Model

from keras_bert import Tokenizer


train = X_train
test  = X_test
label = np.array(y_train)
y_one_train = np_utils.to_categorical(label)

# build tokenizer
dict_path = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
token_dict = {}
with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)     
tokenizer = Tokenizer(token_dict)

# transform label to onehot
# label_encoder = LabelEncoder()
# y = label_encoder.fit_transform(np.array(label))
# y_OneHot = np_utils.to_categorical(y) 

# transform text to indices and segments
indices_train = []
segments_train = []
for text in train:
    indices, segments = tokenizer.encode(first=text, max_len=MAX_SEQ_LEN)
    indices_train.append(indices)
    segments_train.append(segments)

indices_test = []
segments_test = []
for text in test:
    indices, segments = tokenizer.encode(first=text, max_len=MAX_SEQ_LEN)
    indices_test.append(indices)
    segments_test.append(segments)
    
# modeling
base_model = model # pretrain model
non_mask = NonMaskingLayer()(model.output)
conv1 = Convolution1D(256, 3, padding='same')(non_mask)
relu1 = Activation('relu')(conv1)
drop1 = Dropout(0.25)(relu1)
cnn = MaxPool1D(pool_size=4)(drop1)
flat2 = Flatten()(cnn)
drop = Dropout(0.25)(flat2)
fc = Dense(128)(drop)
drop2 = Dropout(0.25)(fc)
fc2 = Dense(64)(drop2)
act = Activation('relu')(fc2)
drop3 = Dropout(0.25)(act)
fc3 = Dense(32)(drop3)
main_output = Dense(units = 7, activation='softmax')(fc3)

cnn_model = Model(base_model.inputs, main_output)
cnn_model.summary()
cnn_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
train_history = cnn_model.fit([indices_train, segments_train], y_one_train, 
                          batch_size = 800, 
                          epochs = 20)

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 30)           0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 30)           0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 30, 768), (2 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 30, 768)      1536        Input-Segment[0][0]              
____________________________________________________________________________________________

Epoch 1/20
15400/15400 [==============================] - 527s 34ms/step - loss: 2.4736 - accuracy: 0.4423
Epoch 2/20
15400/15400 [==============================] - 527s 34ms/step - loss: 1.0619 - accuracy: 0.6460
Epoch 3/20
15400/15400 [==============================] - 526s 34ms/step - loss: 0.9081 - accuracy: 0.6944
Epoch 4/20
15400/15400 [==============================] - 517s 34ms/step - loss: 0.8148 - accuracy: 0.7161
Epoch 5/20
15400/15400 [==============================] - 522s 34ms/step - loss: 0.7591 - accuracy: 0.7311
Epoch 6/20
15400/15400 [==============================] - 541s 35ms/step - loss: 0.7167 - accuracy: 0.7458
Epoch 7/20
15400/15400 [==============================] - 525s 34ms/step - loss: 0.6873 - accuracy: 0.7561
Epoch 8/20
15400/15400 [==============================] - 526s 34ms/step - loss: 0.6595 - accuracy: 0.7685
Epoch 9/20
15400/15400 [==============================] - 524s 34ms/step - loss: 0.6391 - accuracy: 0.7723
Epoch 10/20
15400/15400 [============

In [36]:
predicted = cnn_model.predict([indices_test,segments_test])
bert_predicted = np.argmax(predicted,axis=-1)

In [37]:
from sklearn import metrics
print("Macro_average: {0}".format(metrics.f1_score(y_test, bert_predicted, average="macro")))
print("Micro_average: {0}".format(metrics.f1_score(y_test, bert_predicted, average="micro")))
print(metrics.classification_report(y_test, bert_predicted))

Macro_average: 0.5315134673304331
Micro_average: 0.788034188034188
              precision    recall  f1-score   support

           0       0.74      0.75      0.74       477
           1       0.40      0.35      0.37        23
           2       0.80      0.94      0.86       374
           3       0.62      0.13      0.21       101
           4       0.86      0.93      0.89       662
           5       0.43      0.09      0.15        33
           6       0.54      0.45      0.49        85

    accuracy                           0.79      1755
   macro avg       0.63      0.52      0.53      1755
weighted avg       0.77      0.79      0.77      1755

